In [25]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [17]:
file_path = '/Users/yarden/Documents/Atuda/spring_2024/causal_inference/Causal-Inference-Project-Effect-of-Age-on-Graduating/code/data/processed_data.csv'
data = pd.read_csv(file_path)

data.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,Mother's occupation,...,Admission grade,Displaced,Educational special needs,Gender,Scholarship holder,Unemployment rate,Inflation rate,GDP,Target,Adult
0,1,17,5,171,1,Complete Secondary Education,122.0,Incomplete Secondary Education,Complete Secondary Education,Administrative,...,127.3,1,0,1,0,10.8,1.4,1.74,0,0
1,1,15,1,9254,1,Complete Secondary Education,160.0,Complete Secondary Education,Higher Education - Undergraduate,Professionals,...,142.5,1,0,1,0,13.9,-0.3,0.79,1,0
2,1,1,5,9070,1,Complete Secondary Education,122.0,Other Specific Qualifications,Other Specific Qualifications,Plant and Machine Operators,...,124.8,1,0,1,0,10.8,1.4,1.74,0,0
3,1,17,2,9773,1,Complete Secondary Education,122.0,Other Specific Qualifications,Other Specific Qualifications,Administrative,...,119.6,1,0,0,0,9.4,-0.8,-3.12,1,0
4,2,39,1,8014,0,Complete Secondary Education,100.0,Other Specific Qualifications,Other Specific Qualifications,Plant and Machine Operators,...,141.5,0,0,0,0,13.9,-0.3,0.79,1,1


# Covariate Adjustment

## S-learner

In [18]:
# Choose X (features) and t (treatment variable)
# We'll use a subset of features for simplicity
X = data[['Marital status', 'Application mode', 'Daytime/evening attendance', 'Admission grade',
          'Previous qualification (grade)', 'Unemployment rate', 'Inflation rate', 'GDP']]
t = data['Adult']

# Combine features and treatment variable to create the full feature set
X_full = pd.concat([X, t], axis=1)

# Define the target variable (binary target)
y = data['Target']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.3, random_state=42)

# Fit a Random Forest Classifier model to predict the target variable
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

accuracy

0.5929411764705882

In [19]:
# Make predictions for all data points
y_pred_all = model.predict(X_full)

# Filter predictions where Adult = 1 and Adult = 0
treated_predictions_sum = sum(y_pred_all[data['Adult'] == 1])
control_predictions_sum = sum(y_pred_all[data['Adult'] == 0])

# Calculate the number of data points (n)
n = len(data)

# Calculate the desired value
ATE = (treated_predictions_sum - control_predictions_sum) / n
ATT = treated_predictions_sum / sum(data['Adult'])
ATC = control_predictions_sum / (n - sum(data['Adult']))

print(f'ATE: {ATE}')
print(f'ATT: {ATT}')
print(f'ATC: {ATC}')

ATE: -0.21252059308072488
ATT: 0.3395375070501974
ATC: 0.6078352180936996


## T-learner

In [20]:
X_1 = X[X_full['Adult'] == 1]
X_0 = X[X_full['Adult'] == 0]

y_1 = data[data['Adult'] == 1]['Target']
y_0 = data[data['Adult'] == 0]['Target']

# Split the dataset into training and testing sets for each subgroup
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.3, random_state=42)
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X_0, y_0, test_size=0.3, random_state=42)

# Fit a Random Forest Classifier model to predict the target variable for each subgroup
model_1 = RandomForestClassifier(random_state=42)
model_1.fit(X_train_1, y_train_1)

model_0 = RandomForestClassifier(random_state=42)
model_0.fit(X_train_0, y_train_0)

# Predict on the test set for each subgroup
y_pred_1 = model_1.predict(X_test_1)
y_pred_0 = model_0.predict(X_test_0)

# Calculate accuracy of the model for each subgroup
accuracy_1 = accuracy_score(y_test_1, y_pred_1)
accuracy_0 = accuracy_score(y_test_0, y_pred_0)

accuracy_1, accuracy_0

(0.6052631578947368, 0.6002691790040376)

In [21]:
y_pred_all_1 = model_1.predict(X)
y_pred_all_0 = model_0.predict(X)

treated_predictions_sum = sum(y_pred_all_1)
control_predictions_sum = sum(y_pred_all_0)

ATE = (treated_predictions_sum - control_predictions_sum) / n
ATT = (sum(y_pred_all_1[data['Adult'] == 1]) -  sum(y_pred_all_0[data['Adult'] == 1])) / sum(data['Adult'])
ATC = (sum(y_pred_all_1[data['Adult'] == 0]) -  sum(y_pred_all_0[data['Adult'] == 0])) / (n - sum(data['Adult']))
                                 
print(f'ATE: {ATE}')
print(f'ATT: {ATT}')
print(f'ATC: {ATC}')

ATE: -0.2704165686043775
ATT: -0.2712915961646926
ATC: -0.26978998384491115


# IPW

In [26]:
# Define the treatment variable and features (excluding the treatment)
X = data[['Marital status', 'Application mode', 'Daytime/evening attendance', 'Admission grade',
          'Previous qualification (grade)', 'Unemployment rate', 'Inflation rate', 'GDP']]
treatment = data['Adult']

# Fit a logistic regression model to estimate the propensity score (probability of treatment)
propensity_model = LogisticRegression(random_state=42)
propensity_model.fit(X, treatment)

# Calculate the propensity scores (predicted probabilities of receiving the treatment)
propensity_scores = propensity_model.predict_proba(X)[:, 1]

# Add the propensity scores to the original data
data['propensity_score'] = propensity_scores

/Users/yarden/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [27]:
ATE = sum(data['Target'] * data['Adult'] / data['propensity_score']) / n - sum(data['Target'] * (1 - data['Adult']) / (1 - data['propensity_score'])) / n
ATT = sum(data['Target'] * data['Adult']) / sum(data['Adult']) - sum(data['Target'] * (1 - data['Adult']) * data['propensity_score'] / (1 - data['propensity_score'])) / sum((1 - data['Adult']) * data['propensity_score'] / (1 - data['propensity_score']))
p = sum(data['Adult']) / n
ATC = (ATE - p * ATT) / (1 - p)

print(f'ATE: {ATE}')
print(f'ATT: {ATT}')
print(f'ATC: {ATC}')

ATE: -180.58523060371795
ATT: -0.639306317759285
ATC: -309.439884787484


# Propensity Score Matching

In [28]:
treated = data[data['Adult'] == 1]
control = data[data['Adult'] == 0]

In [29]:
def nearest_neighbor_match(treated, control, treatment_outcome, control_outcome):
    matched_outcomes = []
    
    # Loop over each treated individual
    for i, treated_row in treated.iterrows():
        # Find the control individual with the closest propensity score
        control_row = control.iloc[(control['propensity_score'] - treated_row['propensity_score']).abs().argsort()[:1]]
        
        # Calculate the difference in outcomes
        matched_outcomes.append(treated_row[treatment_outcome] - control_row[control_outcome].values[0])
    
    return matched_outcomes

In [31]:
att_differences = nearest_neighbor_match(treated, control, 'Target', 'Target')
ATT = sum(att_differences) / len(att_differences)

atc_differences = nearest_neighbor_match(control, treated, 'Target', 'Target')
ATC = sum(atc_differences) / len(atc_differences)

p = sum(data['Adult']) / n
ATE = p * ATT + (1 - p) * ATC

print(f'ATE: {ATE}')
print(f'ATT: {ATT}')
print(f'ATC: {ATC}')

ATE: 0.0004706989879971757
ATT: -0.22617033276931753
ATC: 0.16276252019386106
